In [1]:
import numpy as np
from cellpose import models, core
from pathlib import Path
import os,glob
from cellpose import io
import timeit
from omnipose import core as oc
import ncolor.label as nl
import ncolor
import read_roi as rr
import tifffile
import pickle as pk
import momia2 as mo

## Timelapse video XY drift correction

In [1]:
# define functions of drift correction
def xydrift_correction(target_img, shift, max_drift=1000):
    
    if max(np.abs(shift)) <= max_drift:
        return shift_image(target_img, np.array(shift))
    else:
        return target_img
    
def get_xydrift(ref_img, target_img):
    from skimage import registration
    shift, error, _diff = registration.phase_cross_correlation(ref_img, target_img, upsample_factor=10)
    return shift

def shift_image(img, shift):
    from scipy import ndimage as ndi
    """
    correct xy drift between phase contrast image and fluorescent image(s)
    :param img: input image
    :param shift: subpixel xy drift
    :return: drift corrected image
    """
    offset_image = ndi.fourier_shift(np.fft.fftn(img), shift)
    offset_image = np.fft.ifftn(offset_image)
    offset_image = np.round(offset_image.real)
    offset_image[offset_image <= 0] = 1
    # rescaled to avoid int16 overflow
    offset_image[offset_image>=65530] = 65530
    return offset_image.astype(np.uint16)


def correct_tl(pos,softread_data,n_channels=2,ref_channel=1):
    pos_data = {i:[] for i in range(n_channels)}
    output_data = []
    for c in range(n_channels):
        for t in range(softread_data['n_timepoints']):
            pos_data[c].append(mo.io.get_slice_by_index(softread_data,channel=c,position=pos,time=t))
        pos_data[c]=np.array(pos_data[c])
    
    drift_corrected = {i:np.zeros(pos_data[i].shape) for i in range(n_channels)}
    last_drift = np.array([0,0])
    max_drift = 0
    print(len(pos_data[0]))
    for t in range(softread_data['n_timepoints']):
        t_frame = []
        if t == 0:
            for c in range(n_channels):
                t_frame.append(pos_data[c][t])
        else:
            last_drift = last_drift + get_xydrift(pos_data[ref_channel][t-1],pos_data[ref_channel][t])
            max_drift = max(abs(np.max(last_drift)),max_drift)
            for c in range(n_channels):
                t_frame.append(xydrift_correction(pos_data[c][t],last_drift))
        output_data.append(np.array(t_frame))
    output_data = np.array(output_data)
    max_drift = int(round(max_drift))
    cropped = output_data[:,:,max_drift:-max_drift,max_drift:-max_drift]
    return cropped

def correct_tl_simp(pos,timearray,n_channels=1,ref_channel=0):
    pos_data = {i:[] for i in range(n_channels)}
    output_data = []
    for c in range(n_channels):
        for t in range(softread_data['n_timepoints']):
            pos_data[c].append(mo.io.get_slice_by_index(softread_data,channel=c,position=pos,time=t))
        pos_data[c]=np.array(pos_data[c])
    
    drift_corrected = {i:np.zeros(pos_data[i].shape) for i in range(n_channels)}
    last_drift = np.array([0,0])
    max_drift = 0
    print(len(pos_data[0]))
    for t in range(softread_data['n_timepoints']):
        t_frame = []
        if t == 0:
            for c in range(n_channels):
                t_frame.append(pos_data[c][t])
        else:
            last_drift = last_drift + get_xydrift(pos_data[ref_channel][t-1],pos_data[ref_channel][t])
            max_drift = max(abs(np.max(last_drift)),max_drift)
            for c in range(n_channels):
                t_frame.append(xydrift_correction(pos_data[c][t],last_drift))
        output_data.append(np.array(t_frame))
    output_data = np.array(output_data)
    max_drift = int(round(max_drift))
    cropped = output_data[:,:,max_drift:-max_drift,max_drift:-max_drift]
    return cropped

In [ ]:
# batch drift correction
# ignore field #20 due to suboptimal image quality
for f in sorted(glob.glob('/Volumes/JZSSD_temp/20190408_NQTF/*.nd2')):
    if '20.nd2' not in f:
        soft=mo.io.softread_file(f)
        header = f.split('/')[-1].split('.')[0]
        drift_corrected=correct_tl(0,soft,n_channels=2,ref_channel=1)
        tifffile.imwrite('/Volumes/JZSSD_temp/20190408_NQTF/driftcorrected_{}.tif'.format(header), 
                         drift_corrected,imagej=True,
                         metadata={'axes': 'TCYX'})

## Manually define ROIs using Fiji

## Image segmentation using Omnipose
### https://omnipose.readthedocs.io
### PMID: 36253643

In [2]:
models.MODEL_NAMES

['cyto',
 'nuclei',
 'cyto2',
 'bact_phase_cp',
 'bact_fluor_cp',
 'plant_cp',
 'worm_cp',
 'cyto2_omni',
 'bact_phase_omni',
 'bact_fluor_omni',
 'plant_omni',
 'worm_omni',
 'worm_bact_omni',
 'worm_high_res_omni']

In [3]:
model_name = 'bact_phase_omni'
model = models.CellposeModel(gpu=0, model_type=model_name)

2023-01-05 11:01:24,616 [INFO] >>bact_phase_omni<< model set to be used
2023-01-05 11:01:24,618 [INFO] >>>> using CPU


In [59]:
# batch segmentation
verbose = 0 # turn on if you want to see more output 
transparency = True # transparency in flow output
rescale=0.6 # give this a number if you need to upscale or downscale your images
omni = True # we can turn off Omnipose mask reconstruction, not advised 
flow_threshold = 0 # default is .4, but only needed if there are spurious masks to clean up; slows down output
resample = True #whether or not to run dynamics on rescaled grid or original grid 
chans = [0,0]

for f in sorted(glob.glob('/Volumes/JZSSD_temp/20221120_NQTF/pred_results/*_clips.npy')):
    header = f.split('_clips')[0]
    omni_mask_fname = header+'_omni_masks.npy'
    omni_flow_fname = header+'_omni_flows.npy'
    if not os.path.isfile(omni_mask_fname):
        images = np.load(f)
        masks, flows, styles = model.eval([x for x in images],
                                  channels=chans,
                                  rescale=rescale,
                                  transparency=transparency,
                                  flow_threshold=0.3,omni=omni,
                                  resample=resample,verbose=verbose)
        masks = np.array(masks)
        flows = np.array(flows)
        np.save(omni_mask_fname,masks)
        np.save(omni_flow_fname,flows)

2023-01-05 16:37:41,583 [INFO] 100%|##########| 90/90 [02:22<00:00,  1.58s/it]


/var/folders/wk/f42tr5tj0hq3sf4g8x9qng640000gn/T/ipykernel_56659/1336939618.py:22: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  flows = np.array(flows)


2023-01-05 18:27:40,103 [INFO] 69%|######8   | 62/90 [04:11<01:53,  4.06s/it]


KeyboardInterrupt: 